<a href="https://colab.research.google.com/github/RyanChen12035/capstone/blob/main/two_towers_data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Conventional two tower strucutre is for recommanding system and it takes in structured data on the query side (clicking or not, staying time..et al.) and retrieve structured data on the candidate side (recommanded youtube channel, tracks on spotify). They use DNN on both sides and train the two tower model so they are sharing a same embedding space.

In this case, we are aiming for taking in structured data such as prompt here, and retrieve confidential data in the candidate tower. As a result, we meed a BERT model and its embedding to power the vector database. We plan to train a BERT model and set it as the main body of both towers. Extract the embedding from the last layer of the Transformer in the BERT and use it to power FAISS vector database.

Part 1 Data preparation: Set job title as our blacklist. Mix data of job description and medical diagnosis each with 0 and 1 tagging.
Encapsulate it with torch dataset and dataloader, split and shuffle

Part 2 Model and tokenizer: training a text classification model.

Part 3 Extract the embedding space of BERT model and use it to power the vector database.

In [ ]:
!pip install streamlit -q
!pip install PyPDF2 -q
!pip install langchain -q
!pip install sentence-transformers -q
!pip install faiss-cpu
!pip install faiss-gpu

!pip install pydot --quiet
!pip install transformers --quiet
!pip install datasets --quiet

!wget -q -O - ipv4.icanhazip.com

In [62]:
!pip install datasets --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00


In [ ]:
!wget -q -O - ipv4.icanhazip.com

In [ ]:
!huggingface-cli login

In [63]:
import numpy as np
import pandas as pd
from datasets import load_dataset

In [3]:
!pwd

/content


In [82]:
df = pd.read_csv("./fake_job_postings.csv", error_bad_lines=False)
df.head(5)

<ipython-input-82-c4b2bd3a1ca6>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv("./fake_job_postings.csv", error_bad_lines=False)


,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [83]:
def concatenate_cols(row):
    if pd.isna(row['department']):
        row['department'] = 'freelancing'
    if pd.isna(row['salary_range']):
        row['salary_range'] = 'negotiable'
    if pd.isna(row['benefits']):
        row['benefits'] = 'negotiable'
    if len(str(row['company_profile'])) > 200:
        row['company_profile'] = row['company_profile'][:200]
    if len(str(row['description'])) > 200:
        row['description'] = row['description'][:200]
    if len(str(row['requirements'])) > 200:
        row['requirements'] = row['requirements'][:200]


    job_openingline = f"My job is {row['title']} and it's {row['department']}. It locates at {row['location']} and the salary range is {row['salary_range']} "
    job_description = f"Following is the introduction of my company: {row['company_profile']}, the job description: {row['description']}, the requirements: {row['requirements']}, and the benefits: {row['benefits']}"

    return job_openingline + job_description

df['prompt'] = df.apply(lambda row: concatenate_cols(row), axis=1)
df['related_to_job'] = 1
df = df[['prompt', 'related_to_job']]
df.head(10)

,prompt,related_to_job
0,My job is Marketing Intern and it's Marketing....,1
1,My job is Customer Service - Cloud Video Produ...,1
2,My job is Commissioning Machinery Assistant (C...,1
3,My job is Account Executive - Washington DC an...,1
4,My job is Bill Review Manager and it's freelan...,1
5,My job is Accounting Clerk and it's freelancin...,1
6,My job is Head of Content (m/f) and it's ANDRO...,1
7,My job is Lead Guest Service Specialist and...,1
8,My job is HP BSM SME and it's freelancing. It ...,1
9,My job is Customer Service Associate - Part Ti...,1


In [92]:
# download diagnosis data from hugging face and mix them together
# Load a dataset by name from the Hugging Face Hub
symptom_to_diagnosis = load_dataset("gretelai/symptom_to_diagnosis")
symptom_to_diagnosis = symptom_to_diagnosis['train']
df_diagnosis = symptom_to_diagnosis.to_pandas()
df_diagnosis['related_to_job'] = 0
df_diagnosis = df_diagnosis.rename(columns={'input_text': 'prompt'})
df_diagnosis = df_diagnosis[['prompt', 'related_to_job']]
df_diagnosis.head(5)

,prompt,related_to_job
0,I've been having a lot of pain in my neck and ...,0
1,I have a rash on my face that is getting worse...,0
2,I have been urinating blood. I sometimes feel ...,0
3,I have been having trouble with my muscles and...,0
4,I have been feeling really sick. My body hurts...,0


In [81]:
# combine PII by Gemini which is irrelevant to medical diagnosis and diagnosis data. It's better to use medical + PII
df_PII = pd.read_csv("./pii_gemini.csv", error_bad_lines=False)
df_PII.head(5)

<ipython-input-81-63ea109ca68c>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_PII = pd.read_csv("./pii_gemini.csv", error_bad_lines=False)


,Unnamed: 0,Essay,EMAIL,USERNAME,ID_NUM,PHONE_NUM,URL_PERSONAL,STREET_ADDRESS
0,1,"As an essay writer, I am tasked with the respo...","['udavis@hotmail.com', 'jrodriguez@yahoo.com']",['kelleylisa'],[],['+1-839-789-4719'],['https://facebook.com/kellylopez'],"['1122 Megan Squares Suite 848\nPort Jason, TX..."
1,3,"Tyler Lopez, a diligent student with the usern...","['mendozajessica@gmail.com', 'stephenschristin...",['walkerdeborah'],['LNFB87091634579230'],"['+1-622-758-4197x20769', '(942)780-7150x8423']","['https://github.com/emily99', 'https://twitte...","['523 Dana Lane\nJohnsonshire, MT 87296', '142..."
2,5,"Education is the key to a brighter future, and...",['ihays@yahoo.com'],['woodsjohn'],"['146JFQ', '1JN1877']",['351-371-7604x52296'],['https://instagram.com/gonzalezjeffrey'],"['8847 Kramer Station\nSouth Christopher, IN 3..."
3,6,"In today's modern era, the internet has become...",['leachshannon@hotmail.com'],[],"['GDPM43039213765821', 'GB63TPES66875773893055...",['8402752275'],['https://facebook.com/gregorylinda'],"['634 Petersen Orchard\nLawsonbury, PR 03185']"
4,7,My academic journey has been a tapestry woven ...,['pbrown@gmail.com'],['justingreer'],"['057-05-7887', '79-1256E']","['2972670172', '+1-992-751-8536x710']",['https://github.com/montesjennifer'],"['4183 Kimberly Mills\nKatherinemouth, MI 83340']"


In [85]:
df_PII['related_to_job'] = 0
df_PII['prompt'] = df_PII['Essay']
df_PII = df_PII[['prompt', 'related_to_job']]
df_PII.head(5)
print(df_PII.shape[0])

,prompt,related_to_job
0,"As an essay writer, I am tasked with the respo...",0
1,"Tyler Lopez, a diligent student with the usern...",0
2,"Education is the key to a brighter future, and...",0
3,"In today's modern era, the internet has become...",0
4,My academic journey has been a tapestry woven ...,0


In [96]:
print(f"number of job_df {df.shape[0]}")
print(f"number of diagnosis_df {df_diagnosis.shape[0]}")
print(f"number of PII_df {df_PII.shape[0]}")

# in case of imbalanced data
df_job = df.iloc[:6000]
df_final = pd.concat([df_job, df_diagnosis, df_PII])
print(df_final.shape[0])
df_final.head(5)
# 1:2

number of job_df 17880
number of diagnosis_df 853
number of PII_df 2814
9667


,prompt,related_to_job
0,My job is Marketing Intern and it's Marketing....,1
1,My job is Customer Service - Cloud Video Produ...,1
2,My job is Commissioning Machinery Assistant (C...,1
3,My job is Account Executive - Washington DC an...,1
4,My job is Bill Review Manager and it's freelan...,1


In [97]:
df_final[df_final['related_to_job'] == 0]

,prompt,related_to_job
0,I've been having a lot of pain in my neck and ...,0
1,I have a rash on my face that is getting worse...,0
2,I have been urinating blood. I sometimes feel ...,0
3,I have been having trouble with my muscles and...,0
4,I have been feeling really sick. My body hurts...,0
...,...,...
2809,**My Digital Footprint and the Construction of...,0
2810,"Kyle Bennett, a resident of 4183 Frazier Divid...",0
2811,"In the tapestry of human existence, where coun...",0
2812,The intriguing realm of literature has been a ...,0


In [98]:
df_final.to_csv('job_PII_diagnosis.csv', index=False)